# Introdução à Inteligência Artificial 2025/2026

## Projeto 1: O Mundo da Lagarta

### Entrega: 3 de Outubro às 23:59

<left> <img src="figures/lagarta_imagens_easy.png" width="900" /> </center>

## Introdução

Considere o jogo da lagarta, que decorre num mundo simples onde existem apenas paredes e uma maçã, para além da própria lagarta. As paredes rodeiam completamente o mundo (que pode ser quadrado ou retangular) e podem existir também no seu interior. A maçã pode estar num sítio qualquer, apoiada ou a levitar, mas a lagarta está sujeita à força de gravidade: consegue rastejar para o lado quando está apoiada; consegue levantar a cabeça até um certo nível de esforço; se não estiver apoiada, só consegue mover-se para baixo. A cada movimento que faz, a lagarta cresce (a cabeça move-se mas fica corpo onde estava a cabeça) e o seu próprio corpo pode servir de apoio, tal como as paredes. O objetivo é fazer a lagarta chegar à maçã no menor número de movimentos. Neste projeto queremos fazer **apenas a formulação** do problema.

## Representação do Mundo da Lagarta

Considere que cada célula do mundo é dada por um par de coordenadas $(x,y)$, onde **a célula em baixo à esquerda é a célula $(0,0)$** e **a célula no topo à direita é $(M-1,N-1)$**, com $M$ o número de colunas e $N$ o número de linhas do mundo.

Na representação do Mundo da Lagarta considera-se que:
* as paredes são representadas pelo símbolo "="
* a maçã é representada pelo símbolo "x"
* as casas livres são representadas por "."
* a cabeça da lagarta é representada por "@" e o corpo por "o"

Abaixo apresenta-se uma visualização, utilizando os símbolos descritos acima, de um mundo de dimensão 19x8 (o mesmo da primeira figura deste enunciado, imagem central), onde a cabeça da lagarta está na célula (11,3) e a maçã está na célula (16,1). No que se segue do enunciado, para simplificar, "posição da lagarta" significa a posição da sua cabeça.

```
= = = = = = = = = = = = = = = = = = =
= . . . . . . . . . . . . . . . . . =
= . . . . . . . . . . . . . . . . . =
= . . . . . . . . . . . . = . . . . =
= . . . . . . o o o . @ . = . . . . =
= . . . . . . o = o . o = = . . . . =
= . . o o o o o = o o o = = . . x . =
= = = = = = = = = = = = = = = = = = =
```

## Objetivo do Projeto

Formular este problema como um problema de procura no Espaço de Estados, usando a implementação disponibilizada pelo módulo `searchPlus.py` que é fornecido juntamente com este enunciado, assim como `utils.py`. O estado deve ser mínimo, contendo apenas os elementos que mudam com as ações.

Assim, os alunos terão de completar a classe `MundoLagarta`:

```python
from searchPlus import *

# predefinição de um mundo inicial simples:
line1 = "= = = = = = =\n"
line2 = "= x . . . . =\n"
line3 = "= . . . = . =\n"
line4 = "= . . . = . =\n"
line5 = "= = = . = . =\n"
line6 = "= @ . . . . =\n"
line7 = "= = = = = = =\n"
grid = line1 + line2 + line3 + line4 + line5 + line6 + line7

class MundoLagarta(Problem):

    def __init__(self, mundo=grid):
        pass

    def actions (self, state):
        pass

    def result (self, state, action):
        pass

    def goal_test (self, state):
        pass

    def display (self, state):
        pass

    def executa(self, state, actions_list, verbose=False):
        """Executa uma sequência de ações a partir do estado, devolvendo o triplo formado pelo estado 
        final, o custo acumulado e o booleano que indica se o objectivo foi ou não atingido. Se o
        objectivo for atingido antes da sequência ser toda executada, devolve-se o estado e o custo
        corrente. Há o modo verboso e o não verboso, este último selecionado por defeito."""
        cost = 0
        for a in actions_list:
            seg = self.result(state,a)
            cost = self.path_cost(cost,state,a,seg)
            state = seg
            obj = self.goal_test(state)
            if verbose:
                print('Ação:', a)
                print(self.display(state),end='')
                print('Custo Total:',cost)
                print('Atingido o objetivo?', obj)
                print()
            if obj:
                break
        return (state, cost, obj)
```

#### O construtor de `MundoLagarta`

O construtor da classe a desenvolver recebe como input a informação em texto (como exemplificado acima) referente ao mundo inicial, onde as suas dimensões, localização das paredes, da maçã e da lagarta são dados de forma implícita. Será necessário extrair essas informações explicitamente. Repare-se que, no mundo inicial, a lagarta pode ter apenas cabeça e nenhum corpo.

#### O método `actions`

As ações correspondem às direções em que a lagarta se pode movimentar, e são quatro: Esquerda, Direita, Cima e Baixo, identificadas pelos símbolos "E", "D", "C" e "B", respetivamente. Cada movimento leva a uma casa/célula imediatamente adjacente, na ortogonal (não há movimentos na diagonal). Nem todas estas ações são possíveis em todas as situações:

* a lagarta só pode mover-se para casas livres (não pode atravessar paredes nem o seu próprio corpo) ou para a casa onde está a maçã;
* se a cabeça da lagarta não estiver apoiada (a célula imediatamente abaixo está livre) a única ação possível é para baixo (B);
* a ação de levantar a cabeça para cima (C) só é possível se o nível máximo de esforço ainda não tiver sido atingido (nível máximo = 3);
* se a lagarta estiver em esforço (nível de esforço > 0) só pode ir para a direita (D) ou esquerda (E) se isso a colocar numa casa com apoio;
* (uma casa com apoio é uma casa imediatamente acima de uma parede ou do corpo da lagarta).

Note-se que o objetivo é que a lagarta chegue à maçã. Se a lagarta se movimentar de forma a ficar sem ações possíveis, o objetivo do jogo não é atingido. A lista de ações possíveis devolvidas por este método deverá estar **ordenada por ordem alfabética das ações**.

##### Exemplos

**1)** Para o exemplo do mundo simples predefinido acima, a lista de ações do estado inicial é obtido por
```python
p = MundoLagarta()
print(p.actions(p.initial))
```
gerando o output
```python
['D']
```

**2)** Se o mundo inicial for este:
```python
line1 = "= = = = = = = = = = = = = = = = = = =\n"
line2 = "= . . . . . . . . . . . . . . . . . =\n"
line3 = "= . . . . . . o @ . . . . . . . . . =\n"
line4 = "= . . . . o o o = . . . . = . . . . =\n"
line5 = "= . . . . o = = = . . . . = . . . . =\n"
line6 = "= . . . . o o o = . . . = = . . . . =\n"
line7 = "= o o o o o o o = . . . = = . . x . =\n"
line8 = "= = = = = = = = = = = = = = = = = = =\n"
grid2 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8
```
a lista de ações do estado inicial é obtido por
```python
p = MundoLagarta(grid2)
print(p.actions(p.initial))
```
gerando o output
```python
['C', 'D']
```

**3)** E se for este:

```python
line1 = "= = = = = = = = = = = = = = = = = = =\n"
line2 = "= . . . . . . . . . . . . . . . . . =\n"
line3 = "= . . . . . . o o @ . . . . . . . . =\n"
line4 = "= . . . . o o o = . . . . = . . . . =\n"
line5 = "= . . . . o = = = . . . . = . . . . =\n"
line6 = "= . . . . o o o = . . . = = . . . . =\n"
line7 = "= o o o o o o o = . . . = = . . x . =\n"
line8 = "= = = = = = = = = = = = = = = = = = =\n"
grid3 = line1 + line2 + line3 + line4 + line5 + line6 + line7 + line8
```
a única ação possível é...
```python
p = MundoLagarta(grid3)
print(p.actions(p.initial))
```
...ir para baixo
```python
['B']
```

Repare-se que, apesar da força de gravidade, a ação de ir para baixo vai colocar a lagarta na casa imediatamente abaixo de onde está (não cai logo até ao fundo), que neste exemplo seria a casa (9,4).

**4)** Como último exemplo, temos esta situação:
```python
line1 = "= = = = = = =\n"
line2 = "= . . . . . =\n"
line3 = "= . x @ . . =\n"
line4 = "= . . o . . =\n"
line5 = "= . . o . . =\n"
line6 = "= . . o o o =\n"
line7 = "= = = = = = =\n"
grid4 = line1 + line2 + line3 + line4 + line5 + line6 + line7
```
em que a lagarta está em esforço máximo (porque foi para cima 3 vezes consecutivas) e assim a lista de ações é...
```python
p = MundoLagarta(grid4)
print(p.actions(p.initial))
```
...vazia
```python
[]
```


<left> <img src="figures/lagarta_imagem_corpo.png" width="600" /> </center>

#### O método `goal_test`

Chega-se ao objetivo quando a cabeça da lagarta está na mesma casa que a maçã.

#### O método `result`

Repare-se que, se um estado $s$ for argumento do método `result`, ele deve permanecer exatamente igual, não sendo modificado pelo método. Deve ser gerado um novo estado $s´$ e não alterar o estado $s$. Note-se que o teste de validade das ações deve ser feito no método `actions`, sendo ineficiente e redundante repeti-lo no método `result`.

Quando se aplica uma ação a um determinado estado, i.e., quando a lagarta se move numa direção, as seguintes situações podem ocorrer:

- a lagarta aumenta o seu nível de esforço (em 1 unidade), caso a ação seja ir para cima
- a lagarta mantém ou passa a esforço zero, caso contrário

Em qualquer das situações, a cabeça da lagarta muda para a nova posição, e a posição onde estava a cabeça fica com corpo.

#### Igualdade entre estados

É importante que dois estados exatamente com os mesmos valores nos seus atributos sejam considerados iguais mesmo que sejam objetos distintos!

#### Sem programação defensiva

Os alunos apenas precisam de ter em consideração situações iniciais de mundos que sejam válidos: mundos quadrados ou retangulares onde não há símbolos desconhecidos e há sempre uma lagarta (com ou sem corpo) e uma maçã. O principal objetivo desta avaliação é a formulação do problema num espaço de estados. Desenvolva o código assumindo que o input do construtor da classe `MundoLagarta` é sempre válido.

#### Sobre o nível de esforço

A lagarta começa com esforço 0. O nível de esforço aumenta 1 unidade de cada vez que a lagarta se movimenta para cima (independentemente das paredes que possa ou não ter ao seu lado). O nível de esforço só volta a ser 0 se a lagarta se movimentar para uma casa com apoio. Se a lagarta não estava em esforço e se movimenta para uma casa sem apoio (mas não para cima) o nível de esforço não aumenta.

#### Sobre as casas com apoio

Casas com apoio são aquelas que ficam imediatamente acima de uma parede ou do corpo da lagarta, mas com uma exceção: quando a lagarta está em esforço (tem a cabeça levantada porque se movimentou para cima) não se considera que esteja numa casa com apoio, apesar de obviamente ter a cabeça apoiada no corpo.

#### O método `display`

A função `display` pega no estado e faz a visualização do mundo em modo texto, respeitando o formato usado no input do construtor. Um exemplo da sua aplicação para o mundo simples definido anteriormente
```python
p = MundoLagarta()
print(p.display(p.initial))
```
gera o seguinte output
```
= = = = = = =
= x . . . . =
= . . . = . =
= . . . = . =
= = = . = . =
= @ . . . . =
= = = = = = =
```

Quando a cabeça da lagarta está na mesma casa que a maçã (quando se chega ao objetivo), a casa deve mostrar a cabeça da lagarta e não a maçã.

#### O método `executa`

A função `executa` permite executar uma sequência de ações a partir de um determinado estado, devolvendo: o estado que resulta da aplicação da sequência de ações; o custo total dessas ações (cada movimento tem custo 1); a indicação se o objetivo foi ou não satisfeito. Note-se que, se o objetivo for atingido antes de aplicar todas as ações, o processo é interrompido e o método devolve a informação referente ao estado em que o objetivo é atingido.

Há um modo verboso e não verboso, sendo este o modo por defeito. Se se colocar o modo verboso a True, após cada ação é apresentado o estado em modo txt, através do método `display`, juntamente com o custo total até ao momento e a indicação se o objetivo é atingido ou não. Serve para ajudar a testar o código.

Note-se que uma solução para o problema do mundo representado acima, com custo 8, seria:
```python
['D', 'D', 'C', 'C', 'E', 'E', 'C', 'C']
```
Usando o método `executa` podemos testar a implementação do problema. Por exemplo, partindo do mundo inicial previamente definido, e aplicando a lista de ações acima, atinge-se o estado final 
```python
p = MundoLagarta()
seq = ['D', 'D', 'C', 'C', 'E', 'E', 'C', 'C']
p.goal_test(p.executa(p.initial,seq)[0])
```
gera o seguinte output
```python
True
```

## Submissão

#### Quizz

Cada aluno deve completar a implementação da classe `MundoLagarta` e testá-la no link do quizz **Projeto 1** que está na página da disciplina, introduzindo aí o seu código. Pode-se submeter e avaliar o código várias vezes, sendo a submissão com melhor nota a que será considerada. Este projeto deve ser realizado **individualmente**.

O quizz é constituído por uma única pergunta, a implementação da classe `MundoLagarta` e será avaliada através de um conjunto de testes automáticos visíveis e outros invisíveis, valendo um total de 2,15 valores da avaliação da Unidade Curricular.

Os testes visíveis valem 6 em 20, enquanto que os testes invisíveis valem 14 em 20.

#### Ficheiro Pyhton

Simultaneamente, é necessário submeter o script Python que contém todo o código submetido no quizz. **Sem esta submissão o trabalho não poderá ser avaliado**, independentemente do resultado obtido no Moodle. Esse ficheiro deve chamar-se **MundoLagarta_IIA_25_26_alunoXXXXX.py**, em que substituem XXXXX pelo número do aluno.

## **Bom trabalho!**
<left> <img src="figures/lagarta_imagens_impossible.png" width="600" /> </center>

(imagens meramente ilustrativas retiradas e/ou adaptadas de: https://arxiv.org/html/2508.16821v1)
